### MNIST classification

In [30]:
require 'nn';
require 'rnn';
matio = require 'matio'

In [31]:
data = matio.load('ex4data1.mat')
trainset = {}
trainset.data = data.X
trainset.label = data.y[{ {}, 1 }]

In [32]:
setmetatable(trainset,
    {__index = function(t,i)
                return {t.data[i], t.label[i]}
        end}
);
 
function trainset:size()
    return self.data:size(1)
end

In [33]:
mean = {}
stdv = {}
for i=1,400 do
    mean[i] = trainset.data[{ {},{i} }]:mean()
    stdv[i] = trainset.data[{ {}, {i} }]:std()
    --print(i .. 'th mean: ' .. mean[i])
    --print(i .. 'th std dev: ' .. stdv[i])
    trainset.data[{ {},{i} }]:add(-mean[i])
    if stdv[i] ~= 0 then
        trainset.data[{ {},{i} }]:div(stdv[i])
    end
end

### define model

In [34]:
batchSize = 5000
rho = 20
hiddenSize = 10
nIndex = 20
nClass = 10

In [35]:
rnn = nn.Sequential()
r = nn.Recurrent(
    hiddenSize, nn.Linear(nIndex, hiddenSize),
    nn.Linear(hiddenSize, hiddenSize), nn.Sigmoid(),
    rho
)
rnn:add(r)
rnn:add(nn.Linear(hiddenSize, nClass))
rnn:add(nn.LogSoftMax())
rnn = nn.Sequencer(rnn)

In [36]:
criterion = nn.SequencerCriterion(nn.ClassNLLCriterion())

### train

In [37]:
lr = 0.01

In [38]:
prev = 100
for epoch = 1,1e2 do
    local inputs, targets = {}, {}
    for step=1,rho do
        table.insert(inputs, trainset.data[{{},{(step-1)*rho+1, step*rho}}])
        table.insert(targets, trainset.label[{{}}])
    end

    local outputs = rnn:forward(inputs)
    local err = criterion:forward(outputs, targets)
    if epoch%10 == 1 then print(epoch, err/rho) end
    local gradOutputs = criterion:backward(outputs, targets)
    rnn:backward(inputs, gradOutputs)
    rnn:updateParameters(lr)
    rnn:zeroGradParameters()
end

1	2.3557428410779	


11	nan	


21	nan	


31	nan	


41	nan	


51	nan	


61	nan	


71	nan	


81	nan	


91	nan	


Sometimes it works well but most of time it didn't work very well.

### test

In [1]:
correction = {}
--trainsize = trainset:size()
trainsize = 100
for i=10,20 do
    correction[i] = 0
end
for i=1,trainsize do
    local answer = trainset.label[i]
    local inputs = {}
    for step=1,rho do
        table.insert(inputs, trainset.data[{{i},{(step-1)*20+1, step*20}}])
    end
    local prediction = rnn:forward(inputs)
    for d=10,20 do
        guess = prediction[d][{1,{}}]
        local confidences, indices = torch.max(guess)
        -- if i%100 == 1 then print(answer, guess, indices[1]) end
        if (answer == indices) then
            correction[d] = correction[d] + 1
        end
    end
end
for i=10,20 do
    print(i, " = ", correction[i], 100*correction[i]/trainsize .. '%')
end

10	 = 	0	0%	
11	 = 	0	0%	
12	 = 	0	0%	
13	 = 	0	0%	
14	 = 	0	0%	
15	 = 	0	0%	
16	 = 	0	0%	
17	 = 	0	0%	
18	 = 	0	0%	
19	 = 	0	0%	
20	 = 	0	0%	


- Best accuracy: 40%
- Common accuracy: 25~30%